In [19]:
from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

import pyperclip
import time
import datetime

def top_to_bottom(browser, i):
    start = time.time()
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        time.sleep(1.5)

        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height  
        if i:
            if (time.time() - start) > i:
                break
                
def print_csv(f_name, name, contact, city):
    try:
        df = pd.DataFrame({"이름":name,
                           "연락처":contact,
                           "지역":city})
        df = df.fillna('null')
        numbers = df['연락처'].str.split('년').str.get(0).str[-4:]
        int_values = []
        for item in numbers:
            if item.isdigit():
                int_values.append(int(item))
            else:
                int_values.append('없음')
        df.isnull().sum()
        df = df.fillna('null')
        numbers = df['연락처'].str.split('년').str.get(0).str[-4:]
        int_values = []
        for item in numbers:
            if item.isdigit():
                int_values.append(int(item))
            else:
                int_values.append('없음')
        df['출생연도'] = int_values
        df.columns = ['이름', '연락처', '지역', '출생연도']
        df['지역'] = df['지역'].str.split('이전 거주지').str.get(1).str.split('현재').str.get(0).fillna('표시할 장소 없음')
        if len(df[df['연락처'].str.contains('\+82')]) >= 1:
            df['연락처'] = df['연락처'].str.split('\+82 ').str.get(1).str[0:12].fillna('없음')
            df.index[df['연락처'].str.contains('1')]
            phone_num = df.index[df['연락처'].str.contains('1')]
            df.loc[phone_num, '연락처'] = "0" + df.loc[phone_num, '연락처']
        else:
            df['연락처'] = '없음'
        df.to_csv(f"{f_name}.csv", encoding='UTF-8')
    except Exception as ex:
        print(f'{len(name)} {len(contact)} {len(city)}')
        df = pd.DataFrame({"이름":name,
           "연락처":contact,
           "지역":city})
        
        df.to_csv(f"{f_name}.csv", encoding='UTF-8')
        print(ex)

options = webdriver.ChromeOptions()
options.add_argument("--proxy-server=socks5://127.0.0.1:9150")
options.add_argument('window-size=1920x1080')
options.add_argument('disable-gpu')
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_argument('--no-sandbox')
options.add_argument('no-sandox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--start-maximized')
options.add_argument('incognito')

service = Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=service, options=options)

#open browser
browser.get("https://www.facebook.com/")
time.sleep(2)

#log info, search keyword
USER = "shm8485@gmail.com"
PWD = "sohee8485"

#login
elem_id = browser.find_element("id", "email")
pyperclip.copy(USER)
elem_id.send_keys(Keys.CONTROL, "v")
time.sleep(2)

elem_pw = browser.find_element("id", "pass")
pyperclip.copy(PWD)
elem_pw.send_keys(Keys.CONTROL, "v")
elem_pw.send_keys("\n")
time.sleep(3)

#search
keyword = ['여행', '음식', '부동산', '사진', '운동']
kw_city = None
for kw in keyword:
    browser.get(f"https://www.facebook.com/search/groups/?q={kw}")
    time.sleep(2)

    try:
        elem_block = browser.find_element("xpath", "/html/body/div[3]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div[2]/div")
        elem_block.click()
    except:
        pass
    try:
        elem_block = browser.find_element('xpath', '/html/body/div[5]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div[2]/div')
        elem_block.click()
    except:
        pass
    time.sleep(1)


    #show only public groups
    elem_public = browser.find_element('xpath', '/html/body/div[1]/div/div[1]/div/div[3]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div[1]/div[2]/div/div/div[2]/div/div[9]/div[2]/div[2]/div/div[2]/div/input')
    elem_public.click()

    #city
    if kw_city:
        elem_city = browser.find_element("xpath", "/html/body/div[1]/div/div[1]/div/div[3]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div[1]/div[2]/div/div/div[2]/div/div[9]/div[2]/div[1]/div/div/div/div/div/div")
        time.sleep(1)
        elem_city.click()
        elem_city = browser.find_element("xpath", '/html/body/div[1]/div/div[1]/div/div[3]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div[1]/div[2]/div/div/div[2]/div/div[9]/div[2]/div[1]/div/div/div/div/div/div/input')
        time.sleep(1)
        pyperclip.copy(kw_city)
        elem_city.send_keys(Keys.CONTROL, "v")
        time.sleep(1)         
        elem_city = browser.find_element('xpath', '/html/body/div[1]/div/div[1]/div/div[3]/div/div/div/div[2]/div/div/div[1]/div[1]/div/ul/li[1]/div/div[1]/div/div/div')
        elem_city.click()
        time.sleep(1) 

    # top_to_bottom(browser, 5)
    time.sleep(5)

    soup_groups = bs(browser.page_source, "html.parser")
    top_elements = soup_groups.find_all(class_="x1yztbdb")
    groups_href = []

    try:
        for t in top_elements:
            groups_href.append(t.find("a").attrs['href'])
    except Exception as ex:
        print(ex)

    final_href = []

    for g in groups_href:
    #     print(i)
        final_href.append(g[:len(g)-11]+'members')
    # print(len(final_href))


    # full data
    group_name, f_name = [], []
    df_name, df_contact, df_city = [], [], []


    for i in final_href:
        browser.get(i)
        try:
            elem_block = browser.find_element("xpath", "/html/body/div[3]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div[2]/div")
            elem_block.click()
        except:
            pass
        try:
            elem_block = browser.find_element('xpath', '/html/body/div[5]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div[2]/div')
            elem_block.click()
        except:
            pass
        time.sleep(1)
        # scroll to bottom
        top_to_bottom(browser, 2)
        time.sleep(1.5)

        soup_profile = bs(browser.page_source, "html.parser")
        g_name = soup_profile.find('h1')
        group_name.append(g_name.text)
        top_elements = soup_profile.find_all(class_="x1lq5wgf xgqcy7u x30kzoy x9jhf4c x1lliihq")
        profile_list = []
        profile_href = []

        for j in top_elements:
            profile_list.append(j.find("a").attrs['href'])

        for j in profile_list:
            profile_href.append('https://www.facebook.com/profile.php?id='+j[j.index('user/')+5:-1])

        name, contact, city = [], [], []
        error = []
        url = 'php?id='

        for j in profile_href:
            try:
                browser.get(j)
                temp = browser.current_url
                if url in temp:
                    link = j + '&sk='
                else:
                    link = temp[:temp.index('?')]
                browser.get(link+"about_contact_and_basic_info")
                soup_info = bs(browser.page_source, "html.parser")
                soup_name = soup_info.find(class_="x78zum5 x15sbx0n x5oxk1f x1jxijyj xym1h4x xuy2c7u x1ltux0g xc9uqle")
                soup_info_contact = soup_info.find(class_="xyamay9 xqmdsaz x1gan7if x1swvt13")
                t_cont = soup_info_contact.text
                if '\+82 ' in t_cont:
                    contact.append(t_cont)
                    name.append(soup_name.find("h1").text)
                    browser.get(link+"about_places")
                    soup_info = bs(browser.page_source, "html.parser")
                    soup_info_city = soup_info.find(class_="xyamay9 xqmdsaz x1gan7if x1swvt13")
                    city.append(soup_info_city.text)
                else: continue

            except:
                l_name = len(name)
                l_cont = len(contact)
                l_city = len(city)
                print(browser.current_url)
                error.append(browser.current_url)
                if l_name > l_cont:
                    contact.append('이전 거주지표시할 장소 없음')
                    try:
                        browser.get(link+"about_places")
                        time.sleep(1)
                        soup_info = bs(browser.page_source, "html.parser")
                        soup_info_ct = soup_info.find(class_="xyamay9 xqmdsaz x1gan7if x1swvt13")
                        city.append(soup_info_ct.text)
                    except:
                        city.append('')
                elif l_cont > l_city:
                    city.append('')
                else: 
                    continue
                                
        # get real time
        timestamp = datetime.datetime.now()
        t_stamp = timestamp.strftime("%Y-%m-%d_%H%M%S")
        if kw_city:
            t_filename = f'{kw}_{kw_city}_{t_stamp}'
        else:
            t_filename = f'{kw}_{t_stamp}'
        
        print_csv(t_filename, name, contact, city)
        print(f'{g_name}: {t_filename}')
        f_name.append(t_filename)

    timestamp = datetime.datetime.now()
    t_stamp = timestamp.strftime("%Y-%m-%d_%H%M%S")
    if kw_city:
        t_filename = f'{kw}_{kw_city}_filelist_{t_stamp}'
    else:
        t_filename = f'{kw}_filelist_{t_stamp}'


    df = pd.DataFrame({"그룹 이름":group_name,
                       "파일명":f_name})
    df.to_csv(f"{t_filename}.csv", encoding='UTF-8')


https://www.facebook.com/profile.php?id=100094024215589&sk=about_places
https://www.facebook.com/gridanews/about_places
https://www.facebook.com/profile.php?id=105437951527246&sk=about_contact_and_basic_info
https://www.facebook.com/KNK2EMTV/about_contact_and_basic_info
https://www.facebook.com/tripm.kr/about_places
https://www.facebook.com/profile.php?id=100094182000685&sk=about_places
https://www.facebook.com/profile.php?id=100092599136643&sk=about_places
https://www.facebook.com/profile.php?id=100082334500743&sk=about_places
https://www.facebook.com/tico44/about_places
https://www.facebook.com/profile.php?id=100094024215589&sk=about_places
https://www.facebook.com/worldkoreanheritage/about_contact_and_basic_info
https://www.facebook.com/Travelholic24/about_contact_and_basic_info
https://www.facebook.com/buanmasil/about_contact_and_basic_info
https://www.facebook.com/profile.php?id=100094182000685&sk=about_places
https://www.facebook.com/profile.php?id=100064697505607&sk=about_places

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.134)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x005DA813+48355]
	(No symbol) [0x0056C4B1]
	(No symbol) [0x00475358]
	(No symbol) [0x0045D293]
	(No symbol) [0x004BE37B]
	(No symbol) [0x004CC473]
	(No symbol) [0x004BA536]
	(No symbol) [0x004982DC]
	(No symbol) [0x004993DD]
	GetHandleVerifier [0x0083AABD+2539405]
	GetHandleVerifier [0x0087A78F+2800735]
	GetHandleVerifier [0x0087456C+2775612]
	GetHandleVerifier [0x006651E0+616112]
	(No symbol) [0x00575F8C]
	(No symbol) [0x00572328]
	(No symbol) [0x0057240B]
	(No symbol) [0x00564FF7]
	BaseThreadInitThunk [0x768700C9+25]
	RtlGetAppContainerNamedObjectPath [0x77987B4E+286]
	RtlGetAppContainerNamedObjectPath [0x77987B1E+238]


In [109]:
group_name

['맛있는 음식-Tasty food']

In [16]:
def print_csv(f_name, name, contact, city):
    try:
        df = pd.DataFrame({"이름":name,
                           "연락처":contact,
                           "지역":city})
        df = df.fillna('null')
        numbers = df['연락처'].str.split('년').str.get(0).str[-4:]
        int_values = []
        for item in numbers:
            if item.isdigit():
                int_values.append(int(item))
            else:
                int_values.append('없음')
        df.isnull().sum()
        df = df.fillna('null')
        numbers = df['연락처'].str.split('년').str.get(0).str[-4:]
        int_values = []
        for item in numbers:
            if item.isdigit():
                int_values.append(int(item))
            else:
                int_values.append('없음')
        df['출생연도'] = int_values
        df.columns = ['이름', '연락처', '지역', '출생연도']
        df['지역'] = df['지역'].str.split('이전 거주지').str.get(1).str.split('현재').str.get(0).fillna('표시할 장소 없음')
        if len(df[df['연락처'].str.contains('\+82')]) >= 1:
            df['연락처'] = df['연락처'].str.split('\+82 ').str.get(1).str[0:12].fillna('없음')
            df.index[df['연락처'].str.contains('1')]
            phone_num = df.index[df['연락처'].str.contains('1')]
            df.loc[phone_num, '연락처'] = "0" + df.loc[phone_num, '연락처']
        else:
            df['연락처'] = '없음'
        df.to_csv(f"{f_name}.csv", encoding='UTF-8')
    except Exception as ex:
        print(f'{len(name)} {len(contact)} {len(city)}')
        df = pd.DataFrame({"이름":name,
           "연락처":contact,
           "지역":city})
        
        df.to_csv(f"{f_name}.csv", encoding='UTF-8')
        print(ex)

In [17]:
print_csv(t_filename, name, contact, city)

In [65]:
df = pd.DataFrame({"이름":name,
                   "연락처":contact,
                   "지역":city})
df

In [14]:
timestamp = datetime.datetime.now()
t_stamp = timestamp.strftime("%Y-%m-%d_%H%M%S")
if kw_city:
    t_filename = f'여행_{kw_city}_{t_stamp}'
else:
    t_filename = f'여행__{t_stamp}'

In [ ]:
kw

In [91]:
def check_who_has_numb(f_name):
    #번호를 가진 유저 수 확인
    df = pd.read_csv(f'{f_name}.csv', sep=',')
    df2 = df[df['연락처'].str.contains("\+82" , na=False)]
    print(df2)

In [92]:
def main(f_name, keyword=None, city=None):
    start = time.time()
    if keyword == None:
        keyword = f_name
    #crawling(file_name, keyword, city)
    links, name, contact, city, file_name = crawling(f_name, keyword, city)
    print("소요 시간: ", time.time() - start)
    
    
    return links, file_name

In [173]:
if __name__ == "__main__":
    links = main('','부동산')

UnicodeEncodeError: 'cp949' codec can't encode character '\u1ec5' in position 7: illegal multibyte sequence

In [117]:
links, f_name = main('부동산', '부동산', 'seoul')

https://www.facebook.com/groups/1513528458954109/?__tn__=%3C
https://www.facebook.com/groups/seoulrealty/?__tn__=%3C
https://www.facebook.com/groups/575386749142967/?__tn__=%3C
https://www.facebook.com/groups/161421933910183/?__tn__=%3C
https://www.facebook.com/groups/1128887303789381/?__tn__=%3C
https://www.facebook.com/groups/kyounggi/?__tn__=%3C
https://www.facebook.com/groups/248875615574618/?__tn__=%3C
https://www.facebook.com/groups/905507543268022/?__tn__=%3C
https://www.facebook.com/groups/725451924329131/?__tn__=%3C
https://www.facebook.com/groups/453667398384713/?__tn__=%3C
https://www.facebook.com/groups/148673291934658/?__tn__=%3C
https://www.facebook.com/groups/354189351728026/?__tn__=%3C
https://www.facebook.com/groups/914547538933816/?__tn__=%3C
https://www.facebook.com/groups/1983245695289827/?__tn__=%3C
https://www.facebook.com/groups/renetworkgroup/?__tn__=%3C
https://www.facebook.com/groups/337187450092574/?__tn__=%3C
https://www.facebook.com/groups/330732307760715/?

ValueError: not enough values to unpack (expected 4, got 3)

In [22]:
df = pd.DataFrame({"이름":name,
           "연락처":contact,
           "지역":city})
df = df.fillna('null')
numbers = df['연락처'].str.split('년').str.get(0).str[-4:]
int_values = []
for item in numbers:
    if item.isdigit():
        int_values.append(int(item))
    else:
        int_values.append('없음')
df.isnull().sum()
df = df.fillna('null')
numbers = df['연락처'].str.split('년').str.get(0).str[-4:]
int_values = []
for item in numbers:
    if item.isdigit():
        int_values.append(int(item))
    else:
        int_values.append('없음')
df['출생연도'] = int_values
df.columns = ['번호', '이름', '연락처', '지역', '출생연도']
df['지역'] = df['지역'].str.split('이전 거주지').str.get(1).str.split('현재').str.get(0).fillna('표시할 장소 없음')
if len(df[df['연락처'].str.contains('\+82')]) >= 1:
    df['연락처'] = df['연락처'].str.split('\+82 ').str.get(1).str[0:12].fillna('없음')
df.index[df['연락처'].str.contains('1')]
phone_num = df.index[df['연락처'].str.contains('1')]
df.loc[phone_num, '연락처'] = "0" + df.loc[phone_num, '연락처']
df.to_csv(f"{f_name}.csv", encoding='UTF-8')

ValueError: Length mismatch: Expected axis has 4 elements, new values have 5 elements

In [59]:
df = pd.read_csv('여행_2023-06-26_164202.csv')

In [60]:
df = df.fillna('null')
numbers = df['연락처'].str.split('년').str.get(0).str[-4:]
int_values = []
for item in numbers:
    if item.isdigit():
        int_values.append(int(item))
    else:
        int_values.append('없음')
df.isnull().sum()
df = df.fillna('null')
numbers = df['연락처'].str.split('년').str.get(0).str[-4:]
int_values = []
for item in numbers:
    if item.isdigit():
        int_values.append(int(item))
    else:
        int_values.append('없음')
df['출생연도'] = int_values
df.columns = ['번호', '이름', '연락처', '지역', '출생연도']
df['지역'] = df['지역'].str.split('이전 거주지').str.get(1).str.split('현재').str.get(0).fillna('표시할 장소 없음')
if len(df[df['연락처'].str.contains('\+82')]) >= 1:
    df['연락처'] = df['연락처'].str.split('\+82 ').str.get(1).str[0:12].fillna('없음')
if len(df[df['연락처'].str.contains('\+82')]) >= 1:
    df.index[df['연락처'].str.contains('1')]
    phone_num = df.index[df['연락처'].str.contains('1')]
    df.loc[phone_num, '연락처'] = "0" + df.loc[phone_num, '연락처']
else:
    df['연락처'] = '없음'

In [61]:
df

,번호,이름,연락처,지역,출생연도
0,0,최연탄,없음,표시할 장소 없음,없음
1,1,이효린,없음,표시할 장소 없음,없음
2,2,한휘재,없음,인천,없음
3,3,徐載源,없음,서울,없음
4,4,정빈 (JB),없음,서울,2000
5,5,Alejandra Bv (서연),없음,서울,2002
6,6,Kuo Hu,없음,냐짱,없음
7,7,谷本 ゆき,없음,오사카 시,없음
8,8,최영선,없음,표시할 장소 없음,없음
9,9,성장확인,없음,표시할 장소 없음,없음


In [43]:
df = df.fillna('null')
numbers = df['연락처'].str.split('년').str.get(0).str[-4:]
int_values = []

In [44]:
for item in numbers:
    if item.isdigit():
        int_values.append(int(item))
    else:
        int_values.append('없음')

In [45]:
df.isnull().sum()
df = df.fillna('null')
numbers = df['연락처'].str.split('년').str.get(0).str[-4:]
int_values = []

In [46]:
for item in numbers:
    if item.isdigit():
        int_values.append(int(item))
    else:
        int_values.append('없음')

In [47]:
df

,Unnamed: 0,이름,연락처,지역
0,0,최연탄,연락처 정보표시할 연락처 정보 없음웹사이트 및 소셜 링크표시할 링크 없음기본 정보표...,이전 거주지표시할 장소 없음
1,1,이효린,연락처 정보표시할 연락처 정보 없음웹사이트 및 소셜 링크표시할 링크 없음기본 정보여성성별,이전 거주지표시할 장소 없음
2,2,한휘재,연락처 정보표시할 연락처 정보 없음웹사이트 및 소셜 링크표시할 링크 없음기본 정보남성성별,이전 거주지인천현재 거주지
3,3,徐載源,연락처 정보표시할 연락처 정보 없음웹사이트 및 소셜 링크표시할 링크 없음기본 정보남성성별,이전 거주지서울현재 거주지서울출신지
4,4,정빈 (JB),연락처 정보표시할 연락처 정보 없음웹사이트 및 소셜 링크https://asked.k...,이전 거주지서울현재 거주지서울출신지
5,5,Alejandra Bv (서연),연락처 정보표시할 연락처 정보 없음웹사이트 및 소셜 링크표시할 링크 없음기본 정보여...,이전 거주지서울현재 거주지바르셀로나출신지
6,6,Kuo Hu,연락처 정보표시할 연락처 정보 없음웹사이트 및 소셜 링크표시할 링크 없음기본 정보남성성별,이전 거주지냐짱현재 거주지
7,7,谷本 ゆき,연락처 정보표시할 연락처 정보 없음웹사이트 및 소셜 링크표시할 링크 없음기본 정보표...,이전 거주지오사카 시현재 거주지오사카 시출신지
8,8,최영선,연락처 정보표시할 연락처 정보 없음웹사이트 및 소셜 링크표시할 링크 없음기본 정보표...,이전 거주지표시할 장소 없음
9,9,성장확인,연락처 정보표시할 연락처 정보 없음웹사이트 및 소셜 링크표시할 링크 없음기본 정보표...,이전 거주지표시할 장소 없음


In [39]:
df['출생연도'] = int_values
df.columns = ['번호', '이름', '연락처', '지역', '출생연도']
df['지역'] = df['지역'].str.split('이전 거주지').str.get(1).str.split('현재').str.get(0).fillna('표시할 장소 없음')
if len(df[df['연락처'].str.contains('\+82')]) >= 1:
    df['연락처'] = df['연락처'].str.split('\+82 ').str.get(1).str[0:12].fillna('없음')

In [40]:
df.index[df['연락처'].str.contains('1')]
phone_num = df.index[df['연락처'].str.contains('1')]
df.loc[phone_num, '연락처'] = "0" + df.loc[phone_num, '연락처']

In [41]:
df.to_csv("1 여행_2023-06-26_164202.csv", encoding='UTF-8')